In [1]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [42]:
from datasets import load_dataset

dna_ft_dataset = load_dataset("dnagpt/dna_multi_task_finetune")
dna_ft_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 198390
    })
})

In [43]:
data = dna_ft_dataset["train"].train_test_split(train_size=0.9, seed=42)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 178551
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 19839
    })
})

In [4]:
# 初始化tokenizer
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
from transformers import GPT2Tokenizer,GPT2Model,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from tokenizers import Tokenizer
from transformers import GPT2TokenizerFast

tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/dna_eng_bpe")
tokenizer.pad_token = tokenizer.eos_token

2024-10-19 17:40:10.478375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 17:40:10.491047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 17:40:10.505628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 17:40:10.510134: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 17:40:10.521556: I tensorflow/core/platform/cpu_feature_guar

In [5]:
#构建提示词
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text + "\n\n### Response:\n"

#构建提示词
def build_prompt(entry):

    input_data = format_input(entry)

    desired_response = entry['output']

    return input_data + desired_response



In [6]:
example = data["test"][0]
example

{'instruction': 'Determine promoter  detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.',
 'input': 'GGCCCCCGCCGATGCCGGCCATGGTGGAGAAGGGCCCCGAGGTCTCAGGGAAGCGGAGAGGGAGGAACAACGCGGCCGCCTCCGCCTCCGCCGCCGCCGCCTCCGCCGCCGCCTCGGCCGCCTGCGCCTCGCCAGCCGCCACTGCCGCCTCGGGCGCCGCCGCCTCCTCAGCCTCGGCCGCCGCCGCCTCAGCCGCCGCCGCCCCCAATAATGGCCAGAATAAAAGTTTGGCGGCGGCGGCGCCCAATGGCAACAGCAGCAGCAACTCCTGGGAGGAAGGCAGCTCGGGCTCGTCCAGCG',
 'output': 'Non-promoter'}

In [7]:
prompt = build_prompt(example)
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine promoter  detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
GGCCCCCGCCGATGCCGGCCATGGTGGAGAAGGGCCCCGAGGTCTCAGGGAAGCGGAGAGGGAGGAACAACGCGGCCGCCTCCGCCTCCGCCGCCGCCGCCTCCGCCGCCGCCTCGGCCGCCTGCGCCTCGCCAGCCGCCACTGCCGCCTCGGGCGCCGCCGCCTCCTCAGCCTCGGCCGCCGCCGCCTCAGCCGCCGCCGCCCCCAATAATGGCCAGAATAAAAGTTTGGCGGCGGCGGCGCCCAATGGCAACAGCAGCAGCAACTCCTGGGAGGAAGGCAGCTCGGGCTCGTCCAGCG

### Response:
Non-promoter


In [8]:
print('tokens: ', ' '.join(tokenizer.tokenize(prompt)))

tokens:  Bel ow Ġ is Ġ an Ġ instruc tion Ġ th at Ġ describ es Ġ a Ġ t ask . ĠWrit e Ġ a Ġ respon se Ġ th at Ġ appropri at el y Ġ complet es Ġ the Ġ request . Ċ Ċ # # # ĠIn struc tion : Ċ D eter mine Ġ promo ter Ġ Ġ det ection Ġ of Ġ follow ing Ġ d na Ġ sequenc e , ĠTh e Ġ resul t Ġ will Ġ be Ġ on e Ġ of Ġ the Ġ follow ing : ĠN on - promo ter , Ġ promo ter . Ċ Ċ # # # ĠIn put : Ċ GGCCCCC GCCG ATGCC GGCC ATGGTGG AGAAGGG CCCC GAGG TCTCAGGG AAGCGG AGAGGG AGG AACAAC GC GGCCGCC TCCGCC TCCGCC GCCGCC GCC TCCGCC GCCGCC TCGGCC GCCTGC GCC TCGCC AGCCGCC ACTGCC GCC TC GGGCGCC GCCGCC TCCTCAGCC TCGGCC GCCGCC GCCTCAGCC GCCGCC GCCCCC AATAA TGGCC AGAA TAAAAG TTTGGC GGCGGC GGC GCCC AATGGC AAC AGCAGCAGC AACTCC TGGGAGG AAGGC AGCTC GGGC TCGTCC AGCG Ċ Ċ # # # ĠR espon se : Ċ Non - promo ter


In [9]:
def tokenize_function(example):
    prompt =  build_prompt(example)
    result = tokenizer(prompt, padding='max_length', truncation=True, max_length=1024) # max_length=1024
    return result


# Use batched=False for easy
tokenized_datasets = data.map(
    tokenize_function, batched=False,remove_columns=['instruction', 'input', 'output']
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 178551
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 19839
    })
})

In [10]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 178551
})

In [11]:
# 创建DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 因为GPT2是自回归模型，不需要MLM
)

In [12]:
model = GPT2LMHeadModel.from_pretrained("dnagpt/gpt_dna_eng_v1")

In [13]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [40]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
          # return_attention_mask=True,
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    #max_length=max_output_tokens,
    max_new_tokens=8,
  )

  # Decode
  generated_text_with_prompt = tokenizer.decode(generated_tokens_with_prompt[0], skip_special_tokens=True)
  generated_text_answer = generated_text_with_prompt[len(text):]


  return generated_text_answer

# 如果需要进一步清理
def clean_generated_text(text):
    # 去除 'Ġ' 符号并替换为空格
    text = text.replace('Ġ', ' ')
    # 去除多余的空格
    text = ' '.join(text.split())
    return text

In [15]:
input_text = format_input(data["test"][0])

print("input (test):", input_text)

print("--------------------------\n")

print("model's answer: \n")
print(inference(input_text, model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


input (test): Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine promoter  detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
GGCCCCCGCCGATGCCGGCCATGGTGGAGAAGGGCCCCGAGGTCTCAGGGAAGCGGAGAGGGAGGAACAACGCGGCCGCCTCCGCCTCCGCCGCCGCCGCCTCCGCCGCCGCCTCGGCCGCCTGCGCCTCGCCAGCCGCCACTGCCGCCTCGGGCGCCGCCGCCTCCTCAGCCTCGGCCGCCGCCGCCTCAGCCGCCGCCGCCCCCAATAATGGCCAGAATAAAAGTTTGGCGGCGGCGGCGCCCAATGGCAACAGCAGCAGCAACTCCTGGGAGGAAGGCAGCTCGGGCTCGTCCAGCG

### Response:

--------------------------

model's answer: 

 # 1 # # 1 # # 1 # #


In [19]:
training_args = TrainingArguments(
        output_dir='./results_sft',
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=40,
        save_steps=2000,
        save_total_limit=2,
        prediction_loss_only=True,
        #fp16=True, #v100没法用
    )

In [20]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator
)

Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
# 开始训练
trainer.train()

Step,Training Loss
500,1.690500
1000,1.685900
1500,1.674200
2000,1.662800
2500,1.664500
3000,1.665300
3500,1.665000
4000,1.657000
4500,1.658200
5000,1.630300


TrainOutput(global_step=13392, training_loss=1.6280401906659525, metrics={'train_runtime': 31338.5324, 'train_samples_per_second': 17.092, 'train_steps_per_second': 0.427, 'total_flos': 2.79923721633792e+17, 'train_loss': 1.6280401906659525, 'epoch': 3.0})

In [22]:
save_dir = 'gpt_small_sft3/final'
trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: gpt_small_sft3/final


In [23]:
save_dir = 'gpt_small_sft3/final'
finetuned_model = GPT2LMHeadModel.from_pretrained(save_dir, local_files_only=True)

In [32]:
finetuned_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [25]:
print("input (test):", input_text)

print("--------------------------\n")

print("model's answer: \n")
print(inference(input_text, finetuned_model, tokenizer))

print("--------------------------\n")
print("real answer: \n")
print(data["test"][0]["output"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


input (test): Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine promoter  detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
GGCCCCCGCCGATGCCGGCCATGGTGGAGAAGGGCCCCGAGGTCTCAGGGAAGCGGAGAGGGAGGAACAACGCGGCCGCCTCCGCCTCCGCCGCCGCCGCCTCCGCCGCCGCCTCGGCCGCCTGCGCCTCGCCAGCCGCCACTGCCGCCTCGGGCGCCGCCGCCTCCTCAGCCTCGGCCGCCGCCGCCTCAGCCGCCGCCGCCCCCAATAATGGCCAGAATAAAAGTTTGGCGGCGGCGGCGCCCAATGGCAACAGCAGCAGCAACTCCTGGGAGGAAGGCAGCTCGGGCTCGTCCAGCG

### Response:

--------------------------

model's answer: 

promoter, promoter.
promoter.
promoter.
promoter.

--------------------------

real answer: 

Non-promoter


In [ ]:
data["test"]

In [ ]:
#输出全部结果

In [44]:
test_data = data["test"].shuffle(seed=42).select(range(1000))

data_list = []

for entry in test_data:
    input_text = format_input(entry)
    #print(input_text)
    response_text = inference(input_text, finetuned_model, tokenizer)
    #print(response_text)
    data = {
        "instruction":entry["instruction"],
         "input":entry["input"],
         "output":entry["output"],
        "model_response":response_text
    }

    data_list.append(data)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [45]:
import json

# 定义输出文件路径
output_file = 'gpt2-small2-sft3.json'

# 将 Dataset 对象导出为 JSON 文件
# test_data.to_json(output_file)
with open(output_file, "w") as file:
    json.dump(data_list, file, indent=4)  # "indent" for pretty-printing

In [46]:
" ".join(tokenizer.tokenize("promoter"))

'promo ter'

In [49]:
import json
from tqdm import tqdm



with open(output_file, "r") as file:
    test_data = json.load(file)

all_num = len(test_data)
right_sum = 0
same_sum = 0
for item in test_data:
    output = item["output"]
    #output = " ".join(tokenizer.tokenize(output))
    model_response = item["model_response"]
    if model_response == output: #same it
        same_sum = same_sum + 1
        
    if output.find("Non")==-1: # no Non
        if model_response.find(output)!=-1 and model_response.find("Non")==-1: #find it, but no Non
            right_sum = right_sum + 1
    else:
        if model_response.find(output)!=-1: #find it
            right_sum = right_sum + 1


print("presicion", right_sum/all_num, "same", same_sum/all_num)


presicion 0.838 same 0.05
